In [1]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [2]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(i))
    ides.append(preprocessed_ide(i))

In [3]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0
# 21 is the max length
for i in dialogue_input:
    maxLength = max(maxLength, len(i))

In [4]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# Trying padding and processing
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# sample = scaler.fit_transform(sample)
print(len(one_hot_encoded_dialogue_output))
one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)

for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)


0


In [5]:
from keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision



def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=10):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print(len(y_train), len(X_train), len(y_test), len(X_test))
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
#         return the score for more statistics-in the future reference
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])
    print(scores)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = np.array(dialogue_output)

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 


1872 1872 208 208
7/7 - 0s - loss: 40.4752 - accuracy: 0.2692 - 178ms/epoch - 25ms/step
1872 1872 208 208
7/7 - 0s - loss: 49.1752 - accuracy: 0.2548 - 31ms/epoch - 4ms/step
1872 1872 208 208
7/7 - 0s - loss: 46.2653 - accuracy: 0.2596 - 24ms/epoch - 3ms/step
1872 1872 208 208
7/7 - 0s - loss: 48.2247 - accuracy: 0.2452 - 24ms/epoch - 3ms/step
1872 1872 208 208
7/7 - 0s - loss: 47.6030 - accuracy: 0.3077 - 25ms/epoch - 4ms/step
1872 1872 208 208
7/7 - 0s - loss: 58.2581 - accuracy: 0.3125 - 33ms/epoch - 5ms/step
1872 1872 208 208
7/7 - 0s - loss: 66.9880 - accuracy: 0.2740 - 25ms/epoch - 4ms/step
1872 1872 208 208
7/7 - 0s - loss: 72.3224 - accuracy: 0.2404 - 24ms/epoch - 3ms/step
1872 1872 208 208
7/7 - 0s - loss: 59.2889 - accuracy: 0.2548 - 24ms/epoch - 3ms/step
1872 1872 208 208
7/7 - 0s - loss: 60.9021 - accuracy: 0.2788 - 24ms/epoch - 3ms/step
[0.26923078298568726, 0.2548076808452606, 0.25961539149284363, 0.24519230425357819, 0.3076923191547394, 0.3125, 0.2740384638309479, 0.2403

In [6]:
print(X[0], y[0])

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 